# Fashin Mnist With CNN

In [11]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# Importing data from keras datasets

In [22]:
(x_train_full,y_train_full),(x_test_full,y_test_full) = keras.datasets.fashion_mnist.load_data()

# Checking the data size and spiltting to train ,test, validation

In [23]:
print(x_train_full.shape,y_train_full.shape)

(60000, 28, 28) (60000,)


In [39]:
x_train,x_valid = x_train_full[:-35000], x_train_full[-5000:] # x_train contains first 25000 records and x_valid contains last 5000 records
y_train,y_valid = y_train_full[:-35000], y_train_full[-5000:]

x_mean = x_train.mean(axis = 0, keepdims = True)
x_std = x_train.std(axis = 0, keepdims = True)


# Standardising the data

In [40]:
x_train = (x_train - x_mean)/x_std
x_valid = (x_valid - x_mean)/x_std
x_test = (x_test_full - x_mean)/x_std

# Since my x_train is a 3 d array/tensor but my algorith needs an 4d tensor viz. Batch size, image y,image x, number of channels
# Adding a new axis

In [41]:
x_train = x_train[...,np.newaxis]
x_valid = x_valid[...,np.newaxis]
x_test = x_test[...,np.newaxis]

In [42]:
x_train.shape           # shape after adding a new axis

(25000, 28, 28, 1)

# Defining my Convolutional Network

In [87]:
keras.backend.clear_session()
from functools import partial
DefaultConv2d = partial(keras.layers.Conv2D,kernel_size=3, activation = 'relu', padding = 'SAME')

model = keras.models.Sequential([
    DefaultConv2d(filters = 64, kernel_size = 7, input_shape = [28,28,1]),
    keras.layers.MaxPool2D(pool_size = 2),
    DefaultConv2d(filters = 128),
    DefaultConv2d(filters = 128),
    keras.layers.MaxPool2D(pool_size = 2),
    DefaultConv2d(filters = 256),
    DefaultConv2d(filters = 256),
    keras.layers.MaxPool2D(pool_size = 2),
    keras.layers.Flatten(),
    keras.layers.Dense(units = 128, activation = 'relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units = 128, activation = 'relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10,activation = 'softmax')
])

In [88]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [89]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer= 'nadam', metrics = ['accuracy']),
history = model.fit(x_train,y_train, epochs = 5, validation_data=(x_valid,y_valid))
score = model.evaluate(x_test,y_test)

Epoch 1/5
782/782 [==============================] - 11s 12ms/step - loss: 0.8412 - accuracy: 0.6992 - val_loss: 0.5479 - val_accuracy: 0.8144
Epoch 2/5
782/782 [==============================] - 10s 12ms/step - loss: 0.4852 - accuracy: 0.8326 - val_loss: 0.3876 - val_accuracy: 0.8596
Epoch 3/5
782/782 [==============================] - 10s 12ms/step - loss: 0.3992 - accuracy: 0.8605 - val_loss: 0.3868 - val_accuracy: 0.8628
Epoch 4/5
782/782 [==============================] - 10s 12ms/step - loss: 0.3471 - accuracy: 0.8796 - val_loss: 0.3337 - val_accuracy: 0.8804
Epoch 5/5
313/313 [==============================] - 2s 4ms/step - loss: nan - accuracy: 0.0000e+00


In [75]:
x_new = x_train_full[-35000:-34990]
y_new = y_train_full[-35000:-34990]
y_new

array([7, 2, 4, 3, 7, 8, 5, 0, 9, 6], dtype=uint8)

In [76]:
x_new = (x_new - x_mean)/x_std
x_new = x_new[...,np.newaxis]
y_pred = model.predict(x_new)
# y_pred

In [77]:
y_pred[0].argmax()

7

In [78]:
y_pred = [np.array(x).argmax() for x in y_pred]
y_pred

[7, 2, 2, 3, 7, 8, 5, 0, 9, 6]

In [79]:
import pandas as pd
compare = pd.DataFrame({'Original': y_new, 'Predicted': y_pred})
compare

,Original,Predicted
0,7,7
1,2,2
2,4,2
3,3,3
4,7,7
5,8,8
6,5,5
7,0,0
8,9,9
9,6,6


In [83]:
from sklearn.metrics import accuracy_score

In [84]:
score = accuracy_score(y_new, y_pred)
score

0.9